# Compare H151 treated 5xFAD v.s. DMSO treated 5xFAD

In [1]:
suppressPackageStartupMessages({
    library(Seurat)
    library(SeuratWrappers)
    library(patchwork)
    library(ggplot2)
    library(repr)
    library(gridExtra)
    library(edgeR)
    library(SingleCellExperiment)
    library(Matrix)
    library(scran)
    library(tidyverse)
    library(ggrepel)
    library(scater)
})
options(future.globals.maxSize = 1e9)
options(Seurat.object.assay.version = "v5")
options(ggrepel.max.overlaps = Inf)

In [2]:
setwd("~/5XFAD_mouse/data/")

In [3]:
seurat_object <- readRDS(
    "mouseAD_H151_RNA_seurat_object.rds")

In [4]:
unique(seurat_object$genotype)

[1] "WT"    "5XFAD"

In [5]:
wt_object <- subset(seurat_object, subset = genotype == "WT")

In [6]:
unique(wt_object$final_celltype)

[1] DG             CA1            SUB            Oligo          CA2-CA3       
 [6] OPC            Lamp5          Sst            Micro          Pvalb         
[11] Vip            Meis2          Sncg           Astro          Chandelier    
[16] IOL            SMC-Peri       Endo           VLMC           Macro         
[21] Choroid-plexus
21 Levels: DG CA1 CA2-CA3 SUB Lamp5 Pvalb Sst Vip Meis2 Sncg ... Choroid-plexus

In [8]:
for (cluster in unique(wt_object$final_celltype)) {
    curr_object <- wt_object[, wt_object$final_celltype == cluster]
    if (ncol(curr_object) < 10) {
        next
    }
    curr_counts <- LayerData(curr_object, layer = c("counts"))
    curr_meta <- curr_object@meta.data
    curr_sce <- SingleCellExperiment(assays = list(counts = curr_counts), colData = curr_meta)
    curr_sce$group <- factor(curr_sce$group, levels = c("WT_DMSO", "WT_H151"))
    groups <- curr_sce$mouse_id
    curr_aggr <- aggregateAcrossCells(curr_sce, groups, store.number = "group.size")
    
    curr_dge <- DGEList(counts = counts(curr_aggr), group = curr_aggr$group, remove.zeros = TRUE)
    keep <- filterByExpr(curr_dge, min.count = 5, min.prop = 0.5)
    curr_dge <- curr_dge[keep, , keep.lib.sizes=FALSE]
    curr_dge <- calcNormFactors(curr_dge, method = "TMM")
    
    curr_design <- model.matrix(~ 0 + curr_aggr$group)
    colnames(curr_design) <- levels(curr_aggr$group)
    curr_dge <- estimateDisp(curr_dge, curr_design, robust = TRUE)
    curr_fit <- glmQLFit(curr_dge, curr_design)
    curr_glf <- glmQLFTest(curr_fit, contrast = c(-1, 1))
    
    write.table(
        topTags(curr_glf, n = Inf)$table, 
        file.path("DGE_H151_WT/", paste0(cluster, ".result.tsv")), 
        row.names = TRUE,
        col.names = TRUE,
        quote = FALSE,
        sep = "\t"
    )
}

Removing 7840 rows with all zero counts

Removing 7729 rows with all zero counts

Removing 7434 rows with all zero counts

Removing 11031 rows with all zero counts

Removing 8192 rows with all zero counts

Removing 14690 rows with all zero counts

Removing 13268 rows with all zero counts

Removing 12434 rows with all zero counts

Removing 15665 rows with all zero counts

Removing 14013 rows with all zero counts

Removing 15491 rows with all zero counts

Removing 10092 rows with all zero counts

Removing 13475 rows with all zero counts

Removing 17551 rows with all zero counts

Removing 13472 rows with all zero counts

Removing 21379 rows with all zero counts

Removing 22030 rows with all zero counts

Removing 23547 rows with all zero counts

Removing 20806 rows with all zero counts

Removing 22532 rows with all zero counts

Removing 20561 rows with all zero counts



In [6]:
for (cluster in unique(wt_object$final_cluster)) {
    if (!(cluster %in% c("SUB_1", "SUB_2", "SUB_3", "SUB-ProS", "CA2", "CA3"))) {
        next
    }
    curr_object <- wt_object[, wt_object$final_cluster == cluster]
    if (ncol(curr_object) < 10) {
        next
    }
    curr_counts <- LayerData(curr_object, layer = c("counts"))
    curr_meta <- curr_object@meta.data
    curr_sce <- SingleCellExperiment(assays = list(counts = curr_counts), colData = curr_meta)
    curr_sce$group <- factor(curr_sce$group, levels = c("WT_DMSO", "WT_H151"))
    groups <- curr_sce$mouse_id
    curr_aggr <- aggregateAcrossCells(curr_sce, groups, store.number = "group.size")
    
    curr_dge <- DGEList(counts = counts(curr_aggr), group = curr_aggr$group, remove.zeros = TRUE)
    keep <- filterByExpr(curr_dge, min.count = 5, min.prop = 0.5)
    curr_dge <- curr_dge[keep, , keep.lib.sizes=FALSE]
    curr_dge <- calcNormFactors(curr_dge, method = "TMM")
    
    curr_design <- model.matrix(~ 0 + curr_aggr$group)
    colnames(curr_design) <- levels(curr_aggr$group)
    curr_dge <- estimateDisp(curr_dge, curr_design, robust = TRUE)
    curr_fit <- glmQLFit(curr_dge, curr_design)
    curr_glf <- glmQLFTest(curr_fit, contrast = c(-1, 1))
    
    write.table(
        topTags(curr_glf, n = Inf)$table, 
        file.path("DGE_H151_WT/", paste0(cluster, ".result.tsv")), 
        row.names = TRUE,
        col.names = TRUE,
        quote = FALSE,
        sep = "\t"
    )
}

Removing 9213 rows with all zero counts

Removing 9949 rows with all zero counts

Removing 8575 rows with all zero counts

Removing 13620 rows with all zero counts

Removing 12893 rows with all zero counts

Removing 12272 rows with all zero counts

